In [1]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('chapter_4').getOrCreate()

/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


### Q1 Demonstrate how to load a dataset suitable for prediction into a PySpark DataFrame and Display basic statistics and information about the dataset.

In [3]:
data_without_header = spark.read.option("inferSchema", True).option("header", False).csv("covtype.data")
data_without_header.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

In [4]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
colnames = ["Elevation", "Aspect", "Slope", \
"Horizontal_Distance_To_Hydrology", \
"Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",\
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", \
"Horizontal_Distance_To_Fire_Points"] + \
[f"Wilderness_Area_{i}" for i in range(4)] + \
[f"Soil_Type_{i}" for i in range(40)] + \
["Cover_Type"]
data = data_without_header.toDF(*colnames).\
withColumn("Cover_Type",
col("Cover_Type").cast(DoubleType()))
data.head()

Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Wilderness_Area_0=1, Wilderness_Area_1=0, Wilderness_Area_2=0, Wilderness_Area_3=0, Soil_Type_0=0, Soil_Type_1=0, Soil_Type_2=0, Soil_Type_3=0, Soil_Type_4=0, Soil_Type_5=0, Soil_Type_6=0, Soil_Type_7=0, Soil_Type_8=0, Soil_Type_9=0, Soil_Type_10=0, Soil_Type_11=0, Soil_Type_12=0, Soil_Type_13=0, Soil_Type_14=0, Soil_Type_15=0, Soil_Type_16=0, Soil_Type_17=0, Soil_Type_18=0, Soil_Type_19=0, Soil_Type_20=0, Soil_Type_21=0, Soil_Type_22=0, Soil_Type_23=0, Soil_Type_24=0, Soil_Type_25=0, Soil_Type_26=0, Soil_Type_27=0, Soil_Type_28=1, Soil_Type_29=0, Soil_Type_30=0, Soil_Type_31=0, Soil_Type_32=0, Soil_Type_33=0, Soil_Type_34=0, Soil_Type_35=0, Soil_Type_36=0, Soil_Type_37=0, Soil_Type_38=0, Soil_Type_39=0, Cover_Type=5.0)

In [5]:
data.count()

581012

### Q2 Implement a PySpark script to handle missing values and categorical features in the dataset.

In [6]:
data = data.dropna(how="all")

In [7]:
data.count()

581012

In [8]:
(train_data, test_data) = data.randomSplit([0.9, 0.1])
train_data.cache()
test_data.cache()

DataFrame[Elevation: int, Aspect: int, Slope: int, Horizontal_Distance_To_Hydrology: int, Vertical_Distance_To_Hydrology: int, Horizontal_Distance_To_Roadways: int, Hillshade_9am: int, Hillshade_Noon: int, Hillshade_3pm: int, Horizontal_Distance_To_Fire_Points: int, Wilderness_Area_0: int, Wilderness_Area_1: int, Wilderness_Area_2: int, Wilderness_Area_3: int, Soil_Type_0: int, Soil_Type_1: int, Soil_Type_2: int, Soil_Type_3: int, Soil_Type_4: int, Soil_Type_5: int, Soil_Type_6: int, Soil_Type_7: int, Soil_Type_8: int, Soil_Type_9: int, Soil_Type_10: int, Soil_Type_11: int, Soil_Type_12: int, Soil_Type_13: int, Soil_Type_14: int, Soil_Type_15: int, Soil_Type_16: int, Soil_Type_17: int, Soil_Type_18: int, Soil_Type_19: int, Soil_Type_20: int, Soil_Type_21: int, Soil_Type_22: int, Soil_Type_23: int, Soil_Type_24: int, Soil_Type_25: int, Soil_Type_26: int, Soil_Type_27: int, Soil_Type_28: int, Soil_Type_29: int, Soil_Type_30: int, Soil_Type_31: int, Soil_Type_32: int, Soil_Type_33: int, S

In [9]:
data.show(50,vertical=True)

-RECORD 0----------------------------------
 Elevation                          | 2596 
 Aspect                             | 51   
 Slope                              | 3    
 Horizontal_Distance_To_Hydrology   | 258  
 Vertical_Distance_To_Hydrology     | 0    
 Horizontal_Distance_To_Roadways    | 510  
 Hillshade_9am                      | 221  
 Hillshade_Noon                     | 232  
 Hillshade_3pm                      | 148  
 Horizontal_Distance_To_Fire_Points | 6279 
 Wilderness_Area_0                  | 1    
 Wilderness_Area_1                  | 0    
 Wilderness_Area_2                  | 0    
 Wilderness_Area_3                  | 0    
 Soil_Type_0                        | 0    
 Soil_Type_1                        | 0    
 Soil_Type_2                        | 0    
 Soil_Type_3                        | 0    
 Soil_Type_4                        | 0    
 Soil_Type_5                        | 0    
 Soil_Type_6                        | 0    
 Soil_Type_7                    

### Q3 Develop a PySpark script that trains a decision tree model on the training dataset.

In [10]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [11]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [16]:
from pyspark.ml.feature import VectorAssembler
input_cols = colnames[:-1]
vector_assembler = VectorAssembler(inputCols=input_cols,
outputCol="featureVector")
assembled_train_data = vector_assembler.transform(train_data)
assembled_train_data.select("featureVector").show(truncate = False)

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1898.0,34.0,23.0,175.0,56.0,134.0,210.0,184.0,99.0,765.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1901.0,311.0,9.0,30.0,2.0,190.

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier
classifier = DecisionTreeClassifier(seed = 1234, labelCol="Cover_Type",
featuresCol="featureVector",
predictionCol="prediction")
model = classifier.fit(assembled_train_data)
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_2f2c3a21cc9f, depth=5, numNodes=41, numClasses=8, numFeatures=54
  If (feature 0 <= 3047.5)
   If (feature 0 <= 2500.5)
    If (feature 3 <= 15.0)
     If (feature 12 <= 0.5)
      If (feature 23 <= 0.5)
       Predict: 4.0
      Else (feature 23 > 0.5)
       Predict: 3.0
     Else (feature 12 > 0.5)
      Predict: 6.0
    Else (feature 3 > 15.0)
     If (feature 16 <= 0.5)
      Predict: 3.0
     Else (feature 16 > 0.5)
      If (feature 9 <= 1346.0)
       Predict: 3.0
      Else (feature 9 > 1346.0)
       Predict: 4.0
   Else (feature 0 > 2500.5)
    If (feature 17 <= 0.5)
     If (feature 15 <= 0.5)
      Predict: 2.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 17 > 0.5)
     If (feature 0 <= 2711.5)
      Predict: 3.0
     Else (feature 0 > 2711.5)
      If (feature 5 <= 1228.0)
       Predict: 5.0
      Else (feature 5 > 1228.0)
       Predict: 2.0
  Else (feature 0 > 3047.5)
   If (feature 0 <= 3

### Q4 Implement code to evaluate the decision tree model using metrics such as accuracy, precision, and recall.

In [22]:
predictions = model.transform(assembled_train_data)
predictions.select("Cover_Type", "prediction", "probability").\
show(10, truncate = False)

+----------+----------+-----------------------------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                                        |
+----------+----------+-----------------------------------------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,3.550253843149785E-5,0.0676323357120034,0.6079454681009692,0.020697979905563248,0.0015976142294174033,0.30209109951361524,0.0]|
|6.0       |4.0       |[0.0,0.0,0.03732503888024884,0.2379471228615863,0.6174183514774495,0.007776049766718507,0.09953343701399689,0.0]                   |
|6.0       |3.0       |[0.0,3.550253843149785E-5,0.0676323357120034,0.6079454681009692,0.020697979905563248,0.0015976142294174033,0.30209109951361524,0.0]|
|6.0       |3.0       |[0.0,3.550253843149785E-5,0.0676323357120

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type",predictionCol="prediction")
print("accuracy: ",evaluator.setMetricName("accuracy").evaluate(predictions))
print("f1: ",evaluator.setMetricName("f1").evaluate(predictions))
print("recall: ",evaluator.setMetricName("precision").evaluate(predictions))

accuracy:  0.7016466347963157
f1:  0.686596425386393


IllegalArgumentException: MulticlassClassificationEvaluator_79787f5e8ea5 parameter metricName given invalid value precision.

In [26]:
confusion_matrix = predictions.groupBy("Cover_Type").\
pivot("prediction", range(1,8)).count().\
na.fill(0.0).\
orderBy("Cover_Type")
confusion_matrix.show()

+----------+------+------+-----+----+---+---+----+
|Cover_Type|     1|     2|    3|   4|  5|  6|   7|
+----------+------+------+-----+----+---+---+----+
|       1.0|114975| 49537|   94|   0| 20|  6|4965|
|       2.0| 46842|175467| 3225|  48|297| 49| 708|
|       3.0|     0|  3470|24767| 310| 16|106|   0|
|       4.0|     0|     4| 1205|1012|  0|  0|   0|
|       5.0|     0|  6964|  297|  10|377|  0|   0|
|       6.0|     0|  3949| 9374| 128|  9|456|   0|
|       7.0|  7033|    71|    0|   0|  0|  0|9218|
+----------+------+------+-----+----+---+---+----+



In [27]:
from pyspark.sql.functions import col

# Calculate TP, TN, FP, FN for each class
confusion_matrix = confusion_matrix.withColumn("TP", col("1").cast("double"))
for i in range(2, 8):
    confusion_matrix = confusion_matrix.withColumn(f"TN_{i}", col(f"{i}").cast("double"))

# Compute sum of true positives, false positives, and false negatives
confusion_matrix = confusion_matrix.withColumn("sum_TP", sum(col(f"TP") for i in range(1, 8)))
for i in range(2, 8):
    confusion_matrix = confusion_matrix.withColumn(f"sum_FP_{i}", sum(col(f"{j}").cast("double") for j in range(1, 8) if j != i))
    confusion_matrix = confusion_matrix.withColumn(f"sum_FN_{i}", sum(col(f"{i}").cast("double") for j in range(1, 8) if j != i))

# Calculate accuracy, precision, and recall for each class
confusion_matrix = confusion_matrix.withColumn("accuracy", (col("TP") + col("sum_TN")) / (col("TP") + col("sum_TN") + col("sum_FP") + col("sum_FN")))
confusion_matrix = confusion_matrix.withColumn("precision", col("TP") / (col("TP") + col("sum_FP")))
confusion_matrix = confusion_matrix.withColumn("recall", col("TP") / (col("TP") + col("sum_FN")))

# Display the results
confusion_matrix.show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `sum_TN` cannot be resolved. Did you mean one of the following? [`sum_TP`, `sum_FN_2`, `sum_FN_3`, `sum_FN_4`, `sum_FN_5`].;
'Project [Cover_Type#2631, 1#2632L, 2#2633L, 3#2634L, 4#2635L, 5#2636L, 6#2637L, 7#2638L, TP#2905, TN_2#2915, TN_3#2926, TN_4#2938, TN_5#2951, TN_6#2965, TN_7#2980, sum_TP#2996, sum_FP_2#3013, sum_FN_2#3031, sum_FP_3#3050, sum_FN_3#3070, sum_FP_4#3091, sum_FN_4#3113, sum_FP_5#3136, sum_FN_5#3160, ... 5 more fields]
+- Project [Cover_Type#2631, 1#2632L, 2#2633L, 3#2634L, 4#2635L, 5#2636L, 6#2637L, 7#2638L, TP#2905, TN_2#2915, TN_3#2926, TN_4#2938, TN_5#2951, TN_6#2965, TN_7#2980, sum_TP#2996, sum_FP_2#3013, sum_FN_2#3031, sum_FP_3#3050, sum_FN_3#3070, sum_FP_4#3091, sum_FN_4#3113, sum_FP_5#3136, sum_FN_5#3160, ... 4 more fields]
   +- Project [Cover_Type#2631, 1#2632L, 2#2633L, 3#2634L, 4#2635L, 5#2636L, 6#2637L, 7#2638L, TP#2905, TN_2#2915, TN_3#2926, TN_4#2938, TN_5#2